In [ ]:
import musical
import pandas as pd
from pyfaidx import Fasta
import numpy as np
from numpy import dot
from numpy.linalg import norm
import seaborn as sns
import glob
import matplotlib.pyplot as plt
pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       )

In [ ]:
def FormatData(filename, sample_id, catalog, thresh, musical_method, denoising_method, path_to_yields):
    df = pd.read_csv(filename, sep = '\t')
    
    df['Type'] = df['triN'].str[0] + '[' + df['variant'] + ']' + df['triN'].str[2]
    total = df['total_count'].values[0]
    df = df[['Type', 'count']]
    df.columns = ['Type', 'count']
    df = df.set_index('Type')
    df = df.reindex(catalog.index)

    H, model = musical.refit.refit(df, catalog, method = musical_method, thresh=thresh) #WORKS FOR BLADDER v3_3
    
    H['frequency'] = H['count']/np.sum(H['count'])
    H['signature'] = H.index
    H['sample_id'] = sample_id
    
    if denoising_method == 'duplex':
        duplexes = 0
        for x in glob.glob(f'{path_to_yields}/*/*duplex_yield_metrics.txt'):
            y = pd.read_csv(x, sep = '\t')
            duplexes += y[y['fraction']==1]['ds_duplexes'].values[0]
        H['molecule_counts'] = duplexes
            
    elif denoising_method == 'single_strand':
        singlestrands = 0
        for x in glob.glob(f'{path_to_yields}/*/*.step6.r1_consensus_filt_mapped.bam.sscs.counts.txt'):
            with open(x) as f:
                y = f.readlines()[0]
                y = int(y.strip())
                singlestrands+=y
        H['molecule_counts'] = singlestrands/2 #Normalize to duplexes
    elif denoising_method == "umi_agnostic":
        reads = 0
        for x in glob.glob(f'{path_to_yields}/*/*.flagstat'):
            with open(x) as f:
                for line in f:
                    if 'primary mapped' in line:
                        pm = int(line.split(' + ')[0])
                    if 'primary duplicates' in line:
                        dup = int(line.split(' + ')[0])
                        
                reads+=pm-dup
                
        H['molecule_counts'] = reads/2 #Normalize to duplexes
        
    else:
        print("DENOISING METHOD UNKNOWN")
        return(-1)
        

    H['total'] = total
    H['threshold'] = thresh
    H['mode']  = musical_method
    H['denoising'] = denoising_method
    return(H, model, df)


## Melanoma cohort

In [ ]:
catalog = pd.read_csv('../data_objects/sbseverything.txt', sep = '\t')

catalog = catalog.set_index("Type")
catalog = catalog[['SBS7a', 'SBS7b', 'CH']]

mode = 'likelihood_bidirectional'
T = 0.007

results, musical_model, trinucleotide_df = \
        FormatData(\
            path_to_triN, \
            sample_id, \
            catalog, \
            T, \
            mode, \
            denoising_method, \
            path_to_molecule_counts)

results['score'] = results['frequency']*results['total'] / (results['molecule_counts'] * 170 / 2875001522)
print(results)

## Bladder cancer cohort parameters

In [ ]:
catalog = pd.read_csv('../data_objects/sbseverything.txt', sep = '\t')

catalog = catalog.set_index("Type")
catalog = catalog[['SBS2', 'SBS13', 'SBS31', 'SBS35', 'CH']]

mode = 'likelihood_bidirectional'
T = 0.025

# Process as above